In [1]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news")
display(df)

StatementMeta(, 339da275-6933-4a65-a740-e4906f78fe8c, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c7fe3c82-ccee-41c5-9bbb-8aa5800289b7)

In [2]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 339da275-6933-4a65-a740-e4906f78fe8c, 4, Finished, Available, Finished)

In [3]:
#Import the model and configure the input and output columns
model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, 339da275-6933-4a65-a740-e4906f78fe8c, 5, Finished, Available, Finished)

In [4]:
result = model.transform(df)

StatementMeta(, 339da275-6933-4a65-a740-e4906f78fe8c, 6, Finished, Available, Finished)

In [5]:
display(result)

StatementMeta(, 339da275-6933-4a65-a740-e4906f78fe8c, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, bfe23786-296f-4c9c-b72a-6b73493c27e0)

In [6]:
#Create Sentiment Column
from pyspark.sql.functions import col

sentiment_def = result.withColumn("sentiment",col("response.documents.sentiment"))

StatementMeta(, 339da275-6933-4a65-a740-e4906f78fe8c, 8, Finished, Available, Finished)

In [7]:
display(sentiment_def)

StatementMeta(, 339da275-6933-4a65-a740-e4906f78fe8c, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 795b0ffa-b51d-4dea-b232-a0bf32eb13c9)

In [8]:
sentiment_def_final = sentiment_def.drop("error","response")

StatementMeta(, 339da275-6933-4a65-a740-e4906f78fe8c, 10, Finished, Available, Finished)

In [9]:
display(sentiment_def_final)

StatementMeta(, 339da275-6933-4a65-a740-e4906f78fe8c, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 787d586e-4e6a-4402-b93a-d2d47327541d)

In [16]:
from pyspark.sql.functions import col, to_date

sentiment_def_final = sentiment_def_final.withColumn("datePublished", to_date(col("datePublished"),"dd-MMM-yyyy"))

StatementMeta(, 339da275-6933-4a65-a740-e4906f78fe8c, 18, Finished, Available, Finished)

In [10]:
from pyspark.sql.utils import AnalysisException

try:
    
    table_name = 'bing_lake_db.tbl_sentiment_analysis'

    sentiment_def_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    sentiment_def_final.createOrReplaceTempView("vw_sentiment_def_final")

    spark.sql(f""" MERGE INTO {table_name} target_table
                    USING vw_sentiment_def_final source_view 

                    ON source_view.url = target_table.url

                    WHEN MATCHED AND (
                        source_view.title <> target_table.title OR
                        source_view.description <> target_table.description OR
                        source_view.category <> target_table.category OR
                        source_view.image <> target_table.image OR
                        source_view.provider <> target_table.provider OR
                        source_view.datePublished <> target_table.datePublished
                    )
                    THEN UPDATE SET *
                    
                    WHEN NOT MATCHED THEN INSERT *

                   """)

StatementMeta(, 339da275-6933-4a65-a740-e4906f78fe8c, 12, Finished, Available, Finished)

Table Already Exists
